## Data Preparation

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, split=['train'], shuffle_files=True)

TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
TEST_RATIO = 0.2

def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  DATASET_SIZE = len(dataset)
  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

IM_SIZE = 224
def resize_rescale(image, label):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteF1U6VM/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


## Model Subclassing
https://www.tensorflow.org/guide/keras/making_new_layers_and_models_via_subclassing
- create recursively composable layers and models
- auto layers, track the weights and bias of sublayers

In [ ]:
class FeatureExtractor(Layer):
  def __init__(self, filters, kernel_size, strides, padding, activation, pool_size):
    super(FeatureExtractor, self).__init__()

    self.conv1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation)
    self.batch1 = BatchNormalization()
    self.pool1 = MaxPool2D(pool_size=pool_size, strides=strides)

    self.conv2 = Conv2D(filters=filters*2, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation)
    self.batch2 = BatchNormalization()
    self.pool2 = MaxPool2D(pool_size=pool_size, strides=strides)

  def call(self, x, training):
    x = self.conv1(x)
    x = self.batch1(x)
    x = self.pool1(x)

    x = self.conv2(x)
    x = self.batch2(x)
    x = self.pool2(x)

    return x

In [ ]:
feature_sub_classed = FeatureExtractor(8, 3, 1, "valid", "relu", 2)

In [ ]:
func_input = Input(shape=(IM_SIZE, IM_SIZE, 3), name="Input_image")

x = feature_sub_classed(func_input)

x = Flatten()(x)
x = Dense(100, activation='sigmoid')(x)
x = BatchNormalization()(x)
x = Dense(10, activation='sigmoid')(x)
x = BatchNormalization()(x)
func_output = Dense(1, activation='sigmoid')(x)

lenet_model = Model(func_input, func_output, name="Lenet_model")
lenet_model.summary()

Model: "Lenet_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 feature_extractor (FeatureE  (None, 218, 218, 16)     1488      
 xtractor)                                                       
                                                                 
 flatten_4 (Flatten)         (None, 760384)            0         
                                                                 
 dense_12 (Dense)            (None, 100)               76038500  
                                                                 
 batch_normalization_16 (Bat  (None, 100)              400       
 chNormalization)                                                
                                                                 
 dense_13 (Dense)            (None, 10)                

In [ ]:
class LenetModel(Model):
  def __init__(self):
    super(LenetModel, self).__init__()

    self.feature_extractor = FeatureExtractor(8, 3, 1, "valid", "relu", 2)

    self.flatten = Flatten()

    self.dense1 = Dense(100, activation='sigmoid')
    self.batch1 = BatchNormalization()

    self.dense2 = Dense(10, activation='sigmoid')
    self.batch2 = BatchNormalization()

    self.dense3 = Dense(1, activation='sigmoid')

  def call(self, x, training):
    x = self.feature_extractor(x)

    x = self.flatten(x)
    x = self.dense1(x)
    x = self.batch1(x)
    x = self.dense2(x)
    x = self.batch2(x)
    x = self.dense3(x)

    return x

In [ ]:
lenet_sub_classed = LenetModel()
lenet_sub_classed(tf.zeros([1,224,224,3])) # build first
lenet_sub_classed.summary()

Model: "lenet_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_2 (Featur  multiple                 1488      
 eExtractor)                                                     
                                                                 
 flatten_6 (Flatten)         multiple                  0         
                                                                 
 dense_18 (Dense)            multiple                  76038500  
                                                                 
 batch_normalization_24 (Bat  multiple                 400       
 chNormalization)                                                
                                                                 
 dense_19 (Dense)            multiple                  1010      
                                                                 
 batch_normalization_25 (Bat  multiple               

In [ ]:
lenet_sub_classed.compile(optimizer=Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),
              metrics=['accuracy'])
history = lenet_sub_classed.fit(train_dataset, validation_data=val_dataset, epochs=5, verbose=1)

Epoch 1/5
517/517 [==============================] - 55s 94ms/step - loss: 0.6940 - accuracy: 0.5798 - val_loss: 0.7386 - val_accuracy: 0.5661
Epoch 2/5
517/517 [==============================] - 47s 90ms/step - loss: 0.6691 - accuracy: 0.6015 - val_loss: 1.4792 - val_accuracy: 0.5017
Epoch 3/5
517/517 [==============================] - 53s 102ms/step - loss: 0.6996 - accuracy: 0.5155 - val_loss: 0.7020 - val_accuracy: 0.5026
Epoch 4/5
517/517 [==============================] - 47s 91ms/step - loss: 0.6969 - accuracy: 0.5079 - val_loss: 0.8118 - val_accuracy: 0.5044
Epoch 5/5
517/517 [==============================] - 46s 90ms/step - loss: 0.6938 - accuracy: 0.5183 - val_loss: 0.6987 - val_accuracy: 0.4997
